---
title: "Kaggle_exercise_05"
author: "minstar"
date: "2023-05-20"
categories: [python]
---
kaggle 문제풀이 05
https://www.kaggle.com/code/lsiina/exercise-proximity-analysis/edit

# 소개

당신은 위기 대응 팀의 일원으로서 뉴욕시에서 교통사고 대응에 어떻게 병원들이 대응하고 있는지 파악하고자 합니다.

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/wamd0n7.png" width="450"><br/>
</center>

시작하기 전에 아래의 코드 셀을 실행하여 준비를 완료하세요.

In [ ]:
!pip install geopandas

In [2]:
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon

import folium
from folium import Choropleth, Marker
from folium.plugins import HeatMap, MarkerCluster

`embed_map()` 함수를 사용하여 지도를 시각화할 것입니다.

In [3]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# 연습

### 1) 충돌 데이터 시각화하기

아래의 코드 셀을 실행하여 2013-2018년 동안의 주요 차량 충돌을 추적하는 GeoDataFrame인 `collisions`을 로드하세요.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
collisions = gpd.read_file("/content/drive/MyDrive/2023 데이터마이닝/dataset/NYPD_Motor_Vehicle_Collisions/NYPD_Motor_Vehicle_Collisions.shp")
collisions.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET,CROSS STRE,OFF STREET,...,CONTRIBU_2,CONTRIBU_3,CONTRIBU_4,UNIQUE KEY,VEHICLE TY,VEHICLE _1,VEHICLE _2,VEHICLE _3,VEHICLE _4,geometry
0,07/30/2019,0:00,BRONX,10464,40.841100,-73.784960,"(40.8411, -73.78496)",NaN,NaN,121 PILOT STREET,...,Unspecified,NaN,NaN,4180045,Sedan,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,POINT (1043750.211 245785.815)
1,07/30/2019,0:10,QUEENS,11423,40.710827,-73.770660,"(40.710827, -73.77066)",JAMAICA AVENUE,188 STREET,NaN,...,NaN,NaN,NaN,4180007,Sedan,Sedan,NaN,NaN,NaN,POINT (1047831.185 198333.171)
2,07/30/2019,0:25,NaN,NaN,40.880318,-73.841286,"(40.880318, -73.841286)",BOSTON ROAD,NaN,NaN,...,NaN,NaN,NaN,4179575,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,POINT (1028139.293 260041.178)
3,07/30/2019,0:35,MANHATTAN,10036,40.756744,-73.984590,"(40.756744, -73.98459)",NaN,NaN,155 WEST 44 STREET,...,NaN,NaN,NaN,4179544,Box Truck,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,POINT (988519.261 214979.320)
4,07/30/2019,10:00,BROOKLYN,11223,40.600090,-73.965910,"(40.60009, -73.96591)",AVENUE T,OCEAN PARKWAY,NaN,...,NaN,NaN,NaN,4180660,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN,POINT (993716.669 157907.212)


"LATITUDE" 및 "LONGITUDE" 열을 사용하여 충돌 데이터를 시각화하는 대화형 맵을 생성하세요. 어떤 유형의 맵이 가장 효과적인지 어떻게 생각하시나요?

In [7]:
m_1 = folium.Map(location=[40.7, -74], zoom_start=11) 

# Your code here: Visualize the collision data
HeatMap(data=collisions[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m_1)

### 2) 병원 보호범위를 파악하기

다음 코드 셀을 실행하여 병원 데이터를 불러옵니다.

In [9]:
hospitals = gpd.read_file("/content/drive/MyDrive/2023 데이터마이닝/dataset/nyu_2451_34494/nyu_2451_34494.shp")
hospitals.head()

,id,name,address,zip,factype,facname,capacity,capname,bcode,xcoord,ycoord,latitude,longitude,geometry
0,317000001H1178,BRONX-LEBANON HOSPITAL CENTER - CONCOURSE DIVI...,1650 Grand Concourse,10457,3102,Hospital,415,Beds,36005,1008872.0,246596.0,40.843490,-73.911010,POINT (1008872.000 246596.000)
1,317000001H1164,BRONX-LEBANON HOSPITAL CENTER - FULTON DIVISION,1276 Fulton Ave,10456,3102,Hospital,164,Beds,36005,1011044.0,242204.0,40.831429,-73.903178,POINT (1011044.000 242204.000)
2,317000011H1175,CALVARY HOSPITAL INC,1740-70 Eastchester Rd,10461,3102,Hospital,225,Beds,36005,1027505.0,248287.0,40.848060,-73.843656,POINT (1027505.000 248287.000)
3,317000002H1165,JACOBI MEDICAL CENTER,1400 Pelham Pkwy,10461,3102,Hospital,457,Beds,36005,1027042.0,251065.0,40.855687,-73.845311,POINT (1027042.000 251065.000)
4,317000008H1172,LINCOLN MEDICAL & MENTAL HEALTH CENTER,234 E 149 St,10451,3102,Hospital,362,Beds,36005,1005154.0,236853.0,40.816758,-73.924478,POINT (1005154.000 236853.000)


"latitude"와 "longitude" 열을 사용하여 병원 위치를 시각화하세요.

In [10]:
m_2 = folium.Map(location=[40.7, -74], zoom_start=11) 

# Your code here: Visualize the hospital locations
for idx, row in hospitals.iterrows():
    Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(m_2)

### 3) 가장 가까운 병원으로부터 10킬로미터 이상 떨어진 사고는 언제였나요?

가장 가까운 병원으로부터 10킬로미터 이상 떨어진 위치에서 발생한 충돌 사고의 모든 행을 포함하는 DataFrame인 `outside_range`을 생성하세요. 이때, `hospitals`와 `collisions` 모두 좌표 참조 시스템으로 EPSG 2263을 사용하며, EPSG 2263은 미터 단위를 가지고 있습니다.

In [11]:
# Your code here
coverage = gpd.GeoDataFrame(geometry=hospitals.geometry).buffer(10000)
my_union = coverage.geometry.unary_union

outside_range = collisions.loc[~collisions["geometry"].apply(lambda x: my_union.contains(x))]

다음 코드 셀은 가장 가까운 병원으로부터 10킬로미터 이상 떨어진 충돌의 범위를 계산합니다

In [12]:
percentage = round(100*len(outside_range)/len(collisions), 2)
print("Percentage of collisions more than 10 km away from the closest hospital: {}%".format(percentage))

Percentage of collisions more than 10 km away from the closest hospital: 15.12%


### 4) 추천 시스템 만들기

먼 위치에서 충돌이 발생할 때는 부상자를 가장 가까운 이용 가능한 병원으로 이송하는 것이 더욱 중요합니다.

이를 고려하여 다음과 같은 추천 시스템을 만들기로 결정했습니다:
- 충돌 위치 (EPSG 2263에서)를 입력으로 받습니다.
- 가장 가까운 병원을 찾습니다 (거리 계산은 EPSG 2263에서 수행).
- 가장 가까운 병원의 이름을 반환합니다.

In [13]:
def best_hospital(collision_location):
    idx_min = hospitals.geometry.distance(collision_location).idxmin()
    my_hospital = hospitals.iloc[idx_min]
    name = my_hospital["name"]
    return name

# Test your function: this should suggest CALVARY HOSPITAL INC
print(best_hospital(outside_range.geometry.iloc[0]))

CALVARY HOSPITAL INC


### 5) 가장 많은 수요를 받는 병원은 어디인가요?

`outside_range` DataFrame에서만 고려할 때, 가장 추천되는 병원은 어디인가요? 
답변은 4)에서 생성한 함수에 의해 반환된 병원 이름과 정확히 일치하는 Python 문자열이어야 합니다.

In [14]:
# Your code here
highest_demand = outside_range.geometry.apply(best_hospital).value_counts().idxmax()

### 6) 도시는 어디에 새로운 병원을 건설해야 할까요?

다음 코드 셀을 실행하여 병원 위치를 시각화합니다. 이에 추가로, 가장 가까운 병원으로부터 10킬로미터 이상 떨어진 위치에서 발생한 사고도 함께 표시됩니다.

In [15]:
m_6 = folium.Map(location=[40.7, -74], zoom_start=11) 

coverage = gpd.GeoDataFrame(geometry=hospitals.geometry).buffer(10000)
folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m_6)
HeatMap(data=outside_range[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m_6)
folium.LatLngPopup().add_to(m_6)

embed_map(m_6, 'm_6.html')

지도에서 아무 곳이나 클릭하여 해당 위치의 위도와 경도가 표시되는 팝업을 볼 수 있습니다.
뉴욕 시는 3) 단계에서 계산된 백분율을 10% 미만으로 낮추기 위해 두 개의 새로운 병원 위치를 식별하는 데 도움을 요청했습니다. 지도를 사용하여 (지역 법률이나 병원을 건설하기 위해 제거해야 할 잠재적인 건물에 대해 걱정하지 않고) 이 목표를 달성하는 데 도움이 될 수 있는 두 개의 위치를 찾을 수 있을까요?
병원 1의 제안된 위도와 경도를 각각 lat_1과 long_1에 입력해주세요. (병원 2도 마찬가지로 입력해주세요.)
그런 다음 나머지 셀을 그대로 실행하여 새로운 병원의 효과를 확인하세요. 답변이 정답으로 표시될 것입니다. 새로운 두 병원이 백분율을 10% 미만으로 낮추게 됩니다.

In [18]:
# Your answer here: proposed location of hospital 1
lat_1 = 40.6714
long_1 = -73.8492

# Your answer here: proposed location of hospital 2
lat_2 = 40.6702
long_2 = -73.7612

# Do not modify the code below this line
try:
    new_df = pd.DataFrame(
        {'Latitude': [lat_1, lat_2],
         'Longitude': [long_1, long_2]})
    new_gdf = gpd.GeoDataFrame(new_df, geometry=gpd.points_from_xy(new_df.Longitude, new_df.Latitude))
    new_gdf.crs = {'init' :'epsg:4326'}
    new_gdf = new_gdf.to_crs(epsg=2263)
    # get new percentage
    new_coverage = gpd.GeoDataFrame(geometry=new_gdf.geometry).buffer(10000)
    new_my_union = new_coverage.geometry.unary_union
    new_outside_range = outside_range.loc[~outside_range["geometry"].apply(lambda x: new_my_union.contains(x))]
    new_percentage = round(100*len(new_outside_range)/len(collisions), 2)
    print("(NEW) Percentage of collisions more than 10 km away from the closest hospital: {}%".format(new_percentage))
    # Did you help the city to meet its goal?
    q_6.check()
    # make the map
    m = folium.Map(location=[40.7, -74], zoom_start=11) 
    folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m)
    folium.GeoJson(new_coverage.geometry.to_crs(epsg=4326)).add_to(m)
    for idx, row in new_gdf.iterrows():
        Marker([row['Latitude'], row['Longitude']]).add_to(m)
    HeatMap(data=new_outside_range[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m)
    folium.LatLngPopup().add_to(m)
    display(embed_map(m, 'q_6.html'))
except:
    print("error.")
    #q_6.hint()

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


(NEW) Percentage of collisions more than 10 km away from the closest hospital: 9.12%
error.
